# Algoritmos genéticos
--------------------

Tutorial completo: http://deap.readthedocs.io/en/master/tutorials/basic/part1.html

Referencia DEAP: http://deap.readthedocs.io/en/master/api/tools.html

In [1]:
import random
from deap import base, creator, tools

## Fitness

La clase "Fitness" proporcionada es una **clase abstracta** que necesita un atributo de pesos para ser funcional. Un "fitness" a minimizar se construye utilizando pesos negativos, mientras que para maximizar debemos coloar pesos positivos. Es posible que la función de fitness incluya varias funciones internas donde unas deban maximizarse y otras minimizarse. Por esta razón el parámetro "weights" es una tupla.

La función *create()* tiene al menos dos argumentos, un nombre para la clase recién creada y una clase base. Cualquier argumento subsiguiente se convierte en un **atributo de la clase**.

In [2]:
creator.create("FitnessMin", base.Fitness, weights=(1.0,))
# creator.create("FitnessMulti", base.Fitness, weights=(-1.0, 1.0))

## Individuos

El primer individuo que crearemos será una simple lista que contiene flotantes. Para producir este tipo de individuo, necesitamos crear una clase *Individual*, usando el creador, que heredará del tipo de lista estándar y tendrá un atributo fitness.

In [3]:
creator.create("Individual", list, fitness=creator.FitnessMin)

Introducimos ahora el método *register( )*, que toma, al menos, dos argumentos; un alias y una función asignada a este alias. Cualquier otro argumento posterior se pasará a la función cuando esta se invoque. Por lo tanto, el código siguiente creará dos alias en el toolbox; *attr_float* e *individuo*. La primera redirecciona a la función *random.random( )*. El segundo es un acceso directo a la función *initRepeat( )*, fijando su argumento de contenedor a la clase *creator.Individual*, su argumento *func* a la función *toolbox.attr_float()* y su número de repeticiones al argumento *IND_SIZE*.

In [4]:
IND_SIZE=10  # tamaño del individuo. 

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)

Probemos algunas cosas...

In [5]:
i = toolbox.individual()
print(len(i))
print(i)  # cada individuo es una lista de diez números flotantes.

10
[0.3672844457823412, 0.7790911992939252, 0.45574622978037194, 0.16904132416463236, 0.2744239127071303, 0.9410893523378558, 0.4754702439574243, 0.09891663709382403, 0.3190764957999954, 0.49075535188417974]


In [6]:
print(toolbox.attr_float())  # attr_float es un alias a la función random.random().

0.9282735675429145


## Población

Nuestra población será una lista con tantos individuos como queramos establecer. Para ello, registraremos el método "population" como, de nuevo, un acceso directo a la función *tools.initRepeat*, donde llenaremos una lista "*list*"con individuos "*toolbox.individual*".

In [7]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

Probemos esto...

In [8]:
p = toolbox.population(3)
print("Número de individuos:", len(p))
print(p)  # tenemos 3 individuos con 10 flotantes cada uno.

# Podríamos haber registrado "population" también con un número fijado en la población.
# toolbox.register("population", tools.initRepeat, list, toolbox.individual, 3)

Número de individuos: 3
[[0.37075761918846295, 0.7798677459114293, 0.04809259689682999, 0.11334028575089661, 0.47856333419012387, 0.43428209590986655, 0.430176644960792, 0.48228844302093965, 0.8454204163664236, 0.3454451718076943], [0.03039137595504171, 0.2746483779460943, 0.9793057571370583, 0.42456074312096637, 0.48013695877121343, 0.5441195805006059, 0.8218555436536235, 0.022223486484795885, 0.04217833165970153, 0.7179516078996645], [0.10670056209511869, 0.37936507210992376, 0.6328188895471644, 0.31434857129238103, 0.7981311840197746, 0.9505741900958912, 0.19860223555342194, 0.08540216367100362, 0.9511549931495352, 0.47591695433352754]]


## Operadores

Los operadores son como los inicializadores, excepto que algunos ya están implementados en el módulo "toolbox". Una vez se hayan elegido los operadores adecuados, hay que registrarlos en "toolbox". La función (o funciones) de "fitness" nos las podemos crear nosotros.

In [9]:
def evaluate(individual):  # Nuestra función de fitness será tan solo la suma de los 10 valores del individuo.
    return sum(individual),  # Fíjate en la coma al final, quiere decir que devolvemos una tupla.

toolbox.register("mate", tools.cxTwoPoint)  # Cruce en dos puntos
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

## Algoritmo final

In [10]:
def main(pop):
    
    CXPB, MUTPB, NGEN = 0.5, 0.2, 40

    # Evaluate the entire population
    fitnesses = [toolbox.evaluate(i) for i in pop]
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = [toolbox.clone(i) for i in offspring]

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = [toolbox.evaluate(i) for i in invalid_ind] 
        
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop = offspring

    return pop

In [11]:
# Comprobemos la población al inicio 
pop = toolbox.population(n=50)

for k, i in enumerate(pop):
    total = 0
    for j in i:
        total += j
    print("Ind:",k, "->", total)
    
result = main(pop)

print("-------------------------------")
for k, i in enumerate(result):
    total = 0
    for j in i:
        total += j
    print("Ind:",k, "->", total)

Ind: 0 -> 3.6761174653567723
Ind: 1 -> 5.453372208649969
Ind: 2 -> 4.501968099276737
Ind: 3 -> 5.402575527308875
Ind: 4 -> 6.24469035496378
Ind: 5 -> 5.4205017444121015
Ind: 6 -> 5.660210991262966
Ind: 7 -> 4.246243976749309
Ind: 8 -> 4.517007564575538
Ind: 9 -> 6.258523413283948
Ind: 10 -> 5.728527998069859
Ind: 11 -> 4.0461296145578896
Ind: 12 -> 5.7256297171174735
Ind: 13 -> 5.942693817498009
Ind: 14 -> 4.324489480198404
Ind: 15 -> 5.991394326855755
Ind: 16 -> 5.524316972991932
Ind: 17 -> 4.423108089844838
Ind: 18 -> 3.9921287212008036
Ind: 19 -> 4.942972943825481
Ind: 20 -> 5.295739275334482
Ind: 21 -> 4.030359872155145
Ind: 22 -> 5.428121565620315
Ind: 23 -> 5.403327003038093
Ind: 24 -> 6.914731526654303
Ind: 25 -> 4.611200526999221
Ind: 26 -> 5.723074694005733
Ind: 27 -> 4.260861544095921
Ind: 28 -> 4.9696722237541096
Ind: 29 -> 4.252699168197833
Ind: 30 -> 5.209819097649
Ind: 31 -> 3.704916975743044
Ind: 32 -> 5.245427118184674
Ind: 33 -> 5.8925006765402035
Ind: 34 -> 5.64736771

## Ejercicios

1. Incrementa el número de generaciones y observa cómo cambia el resultado final.
2. Cambia el código para que cada individuo solo tenga 5 flotantes y maximice la suma total de cada individuo.
3. Crea un nuevo algoritmo genético que resuelva el [problema del viajante (TSP)](https://es.wikipedia.org/wiki/Problema_del_viajante). 

Consejos para implementar el TSP:
- Para crear las localizaciones de las ciudades (x,y) puedes hacer uso de números complejos, que ya Python incorpora por defecto. El valor absoluto de la resta de dos ciudades te devuelve la distancia entre ellas.
- Para crear el índice de las ciudades tienes [numpy.random.permutation](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.permutation.html).
- El cruce de dos individuos los puedes hacer mediante la función [deap.tools.cxOrdered](http://deap.readthedocs.io/en/master/api/tools.html#deap.tools.cxOrdered).


